In [1]:
import pandas as pd
import plotly.express as px
from pathlib import Path

# Results postprocessing

In [99]:
# ["louisville", "san_jose", "columbia", "north_birmingham", "jackson_county", "duluth", "hill_district"]
community = 'san_jose' # TODO change community name
fig_dir = Path(".").resolve() / "results" / community
fig_dir.mkdir(exist_ok=True, parents=True)

#data in baseline
up00 = pd.read_parquet(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up00__{community}.parquet', engine='pyarrow')

#data after postprocessing
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")
up_baseline = community_results.loc[community_results["upgrade_name"]=="baseline"]

Total_dwelling_unit_count = up00["sample_weight"].sum()

## Housing characterization distribution

### 1. Vintage

In [45]:
vintage_list = ['<1940', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s']
vintage_figure = ['Pre 1940', '1940-1979', 'Post 1980']
vintage_percentage = []
vintage_percentage_figure = []
for vintage in vintage_list:
    temp= up00.loc[up00['build_existing_model.vintage'] == vintage]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    vintage_percentage.append(temp_count)
    
vintage_percentage_figure.append(vintage_percentage[0])
temp = vintage_percentage[1] + vintage_percentage[2] + vintage_percentage[3]+ vintage_percentage[4]
vintage_percentage_figure.append(temp)
temp = vintage_percentage[5] + vintage_percentage[6] + vintage_percentage[7]+ vintage_percentage[8]
vintage_percentage_figure.append(temp)

fig = px.bar(x=vintage_figure, y=vintage_percentage_figure, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=250, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Construction Year",
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                  yaxis_range=[0,0.7])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "vintage.svg")

### 2. Building Type (have change in second run)

In [47]:
# option 1
building_type_list = list(up00['build_existing_model.geometry_building_type_recs'].unique())
building_type_figure = list(up00['build_existing_model.geometry_building_type_recs'].unique())
building_type_percentage = []
for building_type in building_type_list:
    temp = up00.loc[up00['build_existing_model.geometry_building_type_recs'] == building_type]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    building_type_percentage.append(temp_count)
    
fig = px.bar(x=building_type_figure, y=building_type_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=300, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Building Type",
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                 yaxis_range=[0,0.6])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "building_type_1.svg")

In [48]:
# option 2 colors for renter and owner
type_list =  list(up00['build_existing_model.geometry_building_type_recs'].unique())
type_renter_percentage = []
type_owner_percentage = []
for item in type_list:
    temp = up00.loc[up00['build_existing_model.geometry_building_type_recs'] == item]
    temp_renter = temp.loc[temp['build_existing_model.tenure'] == 'Renter']
    temp_owner = temp.loc[temp['build_existing_model.tenure'] == 'Owner']
    temp_renter_count =temp_renter['sample_weight'].sum()/Total_dwelling_unit_count
    temp_owner_count =temp_owner['sample_weight'].sum()/Total_dwelling_unit_count
    type_renter_percentage.append(temp_renter_count)
    type_owner_percentage.append(temp_owner_count)
data = pd.DataFrame()
data['type'] = type_list
data['Renter'] = type_renter_percentage
data['Owner'] = type_owner_percentage
fig = px.bar(data, x="type", y=["Renter", "Owner"], text_auto=True, color_discrete_sequence = ['rgb(0, 121, 61)', 'rgb(255, 204, 0)'])
fig.update_traces(textfont_size=10, textangle=0, textposition="inside", cliponaxis=False)
fig.update_layout(autosize=False, width=340, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Building Type",
                  legend_title="Tenure",
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                  yaxis_range=[0,0.4])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "building_type_2.svg")

In [49]:
# option 3 colors for income group
type_list =  list(up00['build_existing_model.geometry_building_type_recs'].unique())
type_1_percentage = []
type_2_percentage = []
type_3_percentage = []
type_4_percentage = []
type_5_percentage = []
type_6_percentage = []
type_7_percentage = []
for item in type_list:
    temp = up00.loc[up00['build_existing_model.geometry_building_type_recs'] == item]
    temp_1 = temp.loc[temp['build_existing_model.area_median_income'] == '0-30%']
    temp_2 = temp.loc[temp['build_existing_model.area_median_income'] == '30-60%']
    temp_3 = temp.loc[temp['build_existing_model.area_median_income'] == '60-80%']
    temp_4 = temp.loc[temp['build_existing_model.area_median_income'] == '80-100%']
    temp_5 = temp.loc[temp['build_existing_model.area_median_income'] == '100-120%']
    temp_6 = temp.loc[temp['build_existing_model.area_median_income'] == '120-150%']
    temp_7 = temp.loc[temp['build_existing_model.area_median_income'] == '150%+']
    type_1_percentage.append(temp_1['sample_weight'].sum()/Total_dwelling_unit_count)
    type_2_percentage.append(temp_2['sample_weight'].sum()/Total_dwelling_unit_count)
    type_3_percentage.append(temp_3['sample_weight'].sum()/Total_dwelling_unit_count)
    type_4_percentage.append(temp_4['sample_weight'].sum()/Total_dwelling_unit_count)
    type_5_percentage.append(temp_5['sample_weight'].sum()/Total_dwelling_unit_count)
    type_6_percentage.append(temp_6['sample_weight'].sum()/Total_dwelling_unit_count)
    type_7_percentage.append(temp_7['sample_weight'].sum()/Total_dwelling_unit_count)
data = pd.DataFrame()
data['type'] = type_list
data['0-30%'] = type_1_percentage
data['30-60%'] = type_2_percentage
data['60-80%'] = type_3_percentage
data['80-100%'] = type_4_percentage
data['100-120%'] = type_5_percentage
data['120-150%'] = type_6_percentage
data['150%+'] = type_7_percentage
fig = px.bar(data, x="type",
             y=["0-30%", "30-60%", '60-80%', '80-100%', '100-120%', '120-150%', '150%+'],
             text_auto=True,
             color_discrete_sequence = ['rgb(0, 121, 61)',
                                        'rgb(255, 204, 0)',
                                        'rgb(130, 187, 39)',
                                        'rgb(0, 168, 226)',
                                        'rgb(255, 153, 51)',
                                        'rgb(0, 51, 153)',
                                        'rgb(204, 255, 102)'])
fig.update_traces(textfont_size=10, textangle=0, textposition="inside", cliponaxis=False)
fig.update_layout(autosize=False, width=400, height=370, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Building Type",
                  legend_title="Area Median Income",
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                  yaxis_range=[0,0.6])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "building_type_3.svg")

### 3. Heating Fuel (have change in second run)

In [50]:
#option 1  build_existing_model.heating_fuel
heating_fuel_list = ['electricity', 'natural_gas', 'propane', 'fuel_oil']
heating_fuel_figure = ['Electricity', 'Natural Gas', 'Propane', 'Fuel Oil']
heating_fuel_consumption = []
for heating_fuel in heating_fuel_list:
    temp = up00[f'report_simulation_output.end_use_{heating_fuel}_heating_m_btu'].sum() 
    heating_fuel_consumption.append(temp)
    
total_energy = sum(heating_fuel_consumption)
heating_fuel_percentage = heating_fuel_consumption/total_energy

fig = px.bar(x=heating_fuel_figure, y=heating_fuel_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=290, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Heating Fuel",
                  xaxis_title="",
                  yaxis_title="Percent of Energy Use",
                 yaxis_range=[0,1])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "heating_fuel_1.svg")

In [51]:
#option 2
type_list = list(up00['build_existing_model.heating_fuel'].unique())
type_percentage = []
for item in type_list:
    temp = up00.loc[up00['build_existing_model.heating_fuel'] == item]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    type_percentage.append(temp_count)
    
fig = px.bar(x=type_list, y=type_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=350, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Heating Fuel",
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                 yaxis_range=[0,0.8])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "heating_fuel_2.svg")

### 4. Cooling Type

In [52]:
cooling_type_list = list(up00['build_existing_model.hvac_cooling_type'].unique())
cooling_type_duct_percentage = []
cooling_type_noduct_percentage = []
for cooling_type in cooling_type_list:
    temp = up00.loc[up00['build_existing_model.hvac_cooling_type'] == cooling_type]
    temp_duct = temp.loc[temp['build_existing_model.hvac_has_ducts'] == 'Yes']
    temp_noduct = temp.loc[temp['build_existing_model.hvac_has_ducts'] == 'No']
    temp_duct_count =temp_duct['sample_weight'].sum()/Total_dwelling_unit_count
    temp_noduct_count =temp_noduct['sample_weight'].sum()/Total_dwelling_unit_count
    cooling_type_duct_percentage.append(temp_duct_count)
    cooling_type_noduct_percentage.append(temp_noduct_count)
data = pd.DataFrame()
data['cooling_type'] = cooling_type_list
data['Ducted'] = cooling_type_duct_percentage
data['Non-ducted'] =cooling_type_noduct_percentage
fig = px.bar(data, x="cooling_type", y=["Ducted", "Non-ducted"], text_auto=True, color_discrete_sequence = ['rgb(0, 121, 61)', 'rgb(255, 204, 0)'])
fig.update_traces(textfont_size=10, textangle=0, textposition="inside", cliponaxis=False)
#fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=340, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Cooling Type",
                  xaxis_title="",
                  legend_title="Type",
                  yaxis_title="Percent of Housing",
                  yaxis_range=[0,0.6])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "cooling_type.svg")


### 5. Envelope Label

In [100]:
# for buildings with frame wall
frame_wall = up00.loc[up00['build_existing_model.geometry_wall_type'].isin(['Wood Frame', 'Steel Frame'])]
total_count = len(frame_wall)

wall_good = frame_wall.loc[frame_wall["wall_rating"]=="code-compliant"]
wall_inadequate = frame_wall.loc[frame_wall["wall_rating"]=="above-average"]
wall_good_count = len(wall_good)/total_count
wall_inadequate_count = len(wall_inadequate)/total_count
wall_poor_count = 1 - wall_good_count - wall_inadequate_count

ceiling_roof_good = frame_wall.loc[frame_wall["ceiling_roof_rating"]=="code-compliant"]
ceiling_roof_inadequate = frame_wall.loc[frame_wall["ceiling_roof_rating"]=="above-average"]
ceiling_roof_good_count = len(ceiling_roof_good)/total_count
ceiling_roof_inadequate_count = len(ceiling_roof_inadequate)/total_count
ceiling_roof_poor_count = 1 -ceiling_roof_good_count - ceiling_roof_inadequate_count

infiltration_good = frame_wall.loc[frame_wall["infiltration_rating"]=="code-compliant"]
infiltration_inadequate = frame_wall.loc[frame_wall["infiltration_rating"]=="above-average"]
infiltration_good_count = len(infiltration_good)/total_count
infiltration_inadequate_count = len(infiltration_inadequate)/total_count
infiltration_poor_count = 1 - infiltration_good_count - infiltration_inadequate_count

combined_envelope_good = frame_wall.loc[frame_wall["combined_envelope_rating"]=="code-compliant"]
combined_envelope_inadequate = frame_wall.loc[frame_wall["combined_envelope_rating"]=="above-average"]
combined_envelope_good_count = len(combined_envelope_good)/total_count
combined_envelope_inadequate_count = len(combined_envelope_inadequate)/total_count
combined_envelope_poor_count = 1 - combined_envelope_good_count - combined_envelope_inadequate_count

data = pd.DataFrame()
data['type'] = ['Wall', 'Ceiling Roof', 'Infiltration', 'Combined Envelope']
data['Good'] = [wall_good_count, ceiling_roof_good_count, infiltration_good_count, combined_envelope_good_count]
data['Inadequate'] = [wall_inadequate_count, ceiling_roof_inadequate_count, infiltration_inadequate_count, combined_envelope_inadequate_count]
data['Poor'] = [wall_poor_count, ceiling_roof_poor_count, infiltration_poor_count, combined_envelope_poor_count]
fig = px.bar(data, y="type", x=["Good", "Inadequate", "Poor"],
             text_auto=True,
             orientation='h',
             color_discrete_sequence = ['rgb(0, 121, 61)', 'rgb(255, 204, 0)', 'rgb(130, 187, 39)'])
fig.update_traces(textfont_size=10, textangle=0, textposition="inside", cliponaxis=False)
fig.update_layout(autosize=False, width=500, height=260, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Envelope Status for Buildings with Frame Wall",
                  legend_title="Envelope Label",
                  yaxis_title="",
                  xaxis_title="Percent of Housing",
                  xaxis_range=[0,1])
fig.layout.xaxis.tickformat = ',.0%'
fig.show()
fig.write_image(fig_dir / "envelope_label_frame_wall.svg")

In [101]:
# for buildings with masonry wall
masonry_wall = up00.loc[up00['build_existing_model.geometry_wall_type'].isin(['Brick', 'Concrete'])]
total_count = len(masonry_wall)

wall_good = masonry_wall.loc[masonry_wall["wall_rating"]=="code-compliant"]
wall_inadequate = masonry_wall.loc[masonry_wall["wall_rating"]=="above-average"]
wall_good_count = len(wall_good)/total_count
wall_inadequate_count = len(wall_inadequate)/total_count
wall_poor_count = 1 - wall_good_count - wall_inadequate_count

ceiling_roof_good = masonry_wall.loc[masonry_wall["ceiling_roof_rating"]=="code-compliant"]
ceiling_roof_inadequate = masonry_wall.loc[masonry_wall["ceiling_roof_rating"]=="above-average"]
ceiling_roof_good_count = len(ceiling_roof_good)/total_count
ceiling_roof_inadequate_count = len(ceiling_roof_inadequate)/total_count
ceiling_roof_poor_count = 1 -ceiling_roof_good_count - ceiling_roof_inadequate_count

infiltration_good = masonry_wall.loc[masonry_wall["infiltration_rating"]=="code-compliant"]
infiltration_inadequate = masonry_wall.loc[masonry_wall["infiltration_rating"]=="above-average"]
infiltration_good_count = len(infiltration_good)/total_count
infiltration_inadequate_count = len(infiltration_inadequate)/total_count
infiltration_poor_count = 1 - infiltration_good_count - infiltration_inadequate_count

combined_envelope_good = masonry_wall.loc[masonry_wall["combined_envelope_rating"]=="code-compliant"]
combined_envelope_inadequate = masonry_wall.loc[masonry_wall["combined_envelope_rating"]=="above-average"]
combined_envelope_good_count = len(combined_envelope_good)/total_count
combined_envelope_inadequate_count = len(combined_envelope_inadequate)/total_count
combined_envelope_poor_count = 1 - combined_envelope_good_count - combined_envelope_inadequate_count

data = pd.DataFrame()
data['type'] = ['Wall', 'Ceiling Roof', 'Infiltration', 'Combined Envelope']
data['Good'] = [wall_good_count, ceiling_roof_good_count, infiltration_good_count, combined_envelope_good_count]
data['Inadequate'] = [wall_inadequate_count, ceiling_roof_inadequate_count, infiltration_inadequate_count, combined_envelope_inadequate_count]
data['Poor'] = [wall_poor_count, ceiling_roof_poor_count, infiltration_poor_count, combined_envelope_poor_count]
fig = px.bar(data, y="type", x=["Good", "Inadequate", "Poor"],
             text_auto=True,
             orientation='h',
             color_discrete_sequence = ['rgb(0, 121, 61)', 'rgb(255, 204, 0)', 'rgb(130, 187, 39)'])
fig.update_traces(textfont_size=10, textangle=0, textposition="inside", cliponaxis=False)
fig.update_layout(autosize=False, width=500, height=260, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Envelope Status for Buildings with Masonry Wall",
                  legend_title="Envelope Label",
                  yaxis_title="",
                  xaxis_title="Percent of Housing",
                  xaxis_range=[0,1])
fig.layout.xaxis.tickformat = ',.0%'
fig.show()
fig.write_image(fig_dir / "envelope_label_masonry_wall.svg")


In [56]:
good = up00.loc[up00["combined_envelope_rating"]=="code-compliant"]
inadequate = up00.loc[up00["combined_envelope_rating"]=="above-average"]

total_count = len(up00)
good_count = len(good)/total_count
inadequate_count = len(inadequate)/total_count
poor_count = 1 - good_count - inadequate_count

fig = px.bar(x=['Code-compliant', 'Above-average', 'Below-average'], y=[good_count, inadequate_count, poor_count], text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=250, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Envelope Status",
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                  yaxis_range=[0,1])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "envelope_lable.svg")

### 6. Top Enduse

In [102]:
up00 = pd.read_parquet(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up00__{community}.parquet', engine='pyarrow')
df2 = up00.filter(regex='report_simulation_output.end_use_')
enduse_list = list(df2.columns.values)
enduse_percentage = []
up00['report_simulation_output.energy_use_total_m_btu'] = up00['report_simulation_output.energy_use_total_m_btu']*up00['sample_weight']
total_energy_use = up00['report_simulation_output.energy_use_total_m_btu'].sum()
for enduse in enduse_list:
    up00[f'{enduse}'] = up00[f'{enduse}']*up00['sample_weight']
    temp = up00[f'{enduse}'].sum()/total_energy_use
    enduse_percentage.append(temp)
    
enduse_and_percentage = pd.DataFrame(
    {'enduse': enduse_list,
     'enduse_percentage': enduse_percentage
    })

In [103]:
enduse_and_percentage = enduse_and_percentage.sort_values(by='enduse_percentage', ascending=False)
top_enduse_and_percentage = enduse_and_percentage.head(4)
top_enduse = top_enduse_and_percentage['enduse'].tolist()
top_enduse = [s.strip('report_simulation_output.end_use_') for s in top_enduse]
top_enduse

['gas_hot_water_m_b',
 'gas_heating_m_b',
 'ctricity_plug_loads_m_b',
 'ctricity_lighting_interior_m_b']

In [104]:
enduse_name = ['Natural Gas for Hot Water', 'Natural Gas for Heating', 'Electricity for Plug Loads', 'Electricity for Interior Lighting']
#'Natural Gas for Hot Water', 'Natural Gas for Heating',
#'Electricity for Cooling', 'Electricity for Heating', 'Electricity for Interior Lighting',  'Electricity for Plug Loads',
fig = px.bar(x=enduse_name, y=top_enduse_and_percentage['enduse_percentage'], text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=310, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Top 4 End Use",
                  xaxis_title="",
                  yaxis_title="Percent of Energy Use",
                  yaxis_range=[0,0.3])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "enduse.svg")

### 7. Energy Burden (have change in second run)

In [61]:
#option 1
metric = "energy_burden"
metric_name = metric.replace("_", " ").title()

type_list = ['<2%', '2-4%', '4-6%', '6-8%', '8-10%', '10%+']
type_percentage = []
for item in type_list:
    temp = up_baseline.loc[up_baseline[metric] == item]
    temp_count =temp['sample_weight'].sum()/up_baseline["sample_weight"].sum()
    type_percentage.append(temp_count)
    
fig = px.bar(x=type_list, y=type_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=350, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title=metric_name,
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                  yaxis_range=[0,0.4])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / f"{metric}_1.svg")

In [62]:
#option 2
metric = "energy_burden"
metric_name = metric.replace("_", " ").title()

type_list = ['<2%', '2-4%', '4-6%', '6-8%', '8-10%', '10%+']
type_percentage = []
for item in type_list:
    temp = up_baseline.loc[up_baseline[metric] == item]
    temp_count =temp['sample_weight'].sum()/up_baseline["sample_weight"].sum()
    type_percentage.append(temp_count)
    
fig = px.bar(x=type_list, y=type_percentage, text_auto=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=350, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title=metric_name,
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                  yaxis_range=[0,0.3])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / f"{metric}_2.svg")

In [63]:
#option 3
metric = "energy_burden"
metric_name = metric.replace("_", " ").title()
type_list =  ['<2%', '2-4%', '4-6%', '6-8%', '8-10%', '10%+']

fuel_1_percentage = []
fuel_2_percentage = []
fuel_3_percentage = []
fuel_4_percentage = []
fuel_5_percentage = []
fuel_6_percentage = []
for item in type_list:
    temp = up_baseline.loc[up_baseline[metric] == item]
    temp_1 = temp.loc[temp['build_existing_model.heating_fuel'] == 'Electricity']
    temp_2 = temp.loc[temp['build_existing_model.heating_fuel'] == 'Natural Gas']
    temp_3 = temp.loc[temp['build_existing_model.heating_fuel'] == 'Propane']
    temp_4 = temp.loc[temp['build_existing_model.heating_fuel'] == 'Fuel Oil']
    temp_5 = temp.loc[temp['build_existing_model.heating_fuel'] == 'Other Fuel']
    temp_6 = temp.loc[temp['build_existing_model.heating_fuel'] == 'None']
    fuel_1_percentage.append(temp_1['sample_weight'].sum()/up_baseline["sample_weight"].sum())
    fuel_2_percentage.append(temp_2['sample_weight'].sum()/up_baseline["sample_weight"].sum())
    fuel_3_percentage.append(temp_3['sample_weight'].sum()/up_baseline["sample_weight"].sum())
    fuel_4_percentage.append(temp_4['sample_weight'].sum()/up_baseline["sample_weight"].sum())
    fuel_5_percentage.append(temp_5['sample_weight'].sum()/up_baseline["sample_weight"].sum())
    fuel_6_percentage.append(temp_6['sample_weight'].sum()/up_baseline["sample_weight"].sum())
data = pd.DataFrame()
data['type'] = type_list
data['Electricity'] = fuel_1_percentage
data['Natural Gas'] = fuel_2_percentage
data['Propane'] = fuel_3_percentage
data['Fuel Oil'] = fuel_4_percentage
data['Other Fuel'] = fuel_5_percentage
data['None'] = fuel_6_percentage

fig = px.bar(data, x="type",
             y=['Electricity', 'Natural Gas', 'Propane', 'Fuel Oil', 'Other Fuel', 'None'],
             text_auto=True,
             color_discrete_sequence = ['rgb(0, 121, 61)',
                                        'rgb(255, 204, 0)',
                                        'rgb(130, 187, 39)',
                                        'rgb(0, 168, 226)',
                                        'rgb(255, 153, 51)',
                                        'rgb(0, 51, 153)'])
fig.update_traces(textfont_size=10, textangle=0, textposition="inside", cliponaxis=False)
fig.update_layout(autosize=False, width=410, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title=metric_name,
                  legend_title="Heating Fuel",
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                  yaxis_range=[0,0.4])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / f"{metric}_3.svg")

### 8. Efficiency of Heating System (have change in second run)

In [65]:
heating_eff_list = up00['build_existing_model.hvac_heating_efficiency'].unique().tolist()
heating_eff_percentage = []
for item in heating_eff_list:
    temp = up00.loc[up00['build_existing_model.hvac_heating_efficiency'] == item]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    heating_eff_percentage.append(temp_count)
    
fig = px.bar(x=heating_eff_list, y=heating_eff_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=700, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Efficiency of Heating Systems",
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                 yaxis_range=[0,0.4])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "heating_efficiency.svg")

### 9. Efficiency of Cooling System (have change in second run)

In [107]:
cooling_eff_list = up00['build_existing_model.hvac_cooling_efficiency'].unique().tolist()
cooling_eff_percentage = []
for item in cooling_eff_list:
    temp = up00.loc[up00['build_existing_model.hvac_cooling_efficiency'] == item]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    cooling_eff_percentage.append(temp_count)
    
fig = px.bar(x=cooling_eff_list, y=cooling_eff_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=500, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Efficiency of Cooling Systems",
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                 yaxis_range=[0,0.8])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "cooling_efficiency.svg")

### 10. Percentage of Housing Available for Each Upgrade (have change in second run)

In [108]:
Total_dwelling_unit = up_baseline["sample_weight"].sum()
type_list = community_results['upgrade_name'].unique().tolist()
type_list.remove('baseline')
type_percentage = []
for item in type_list:
    temp = community_results.loc[community_results["upgrade_name"] == item]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit
    type_percentage.append(temp_count)
    
fig = px.bar(x=type_list, y=type_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=900, height=400, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Applicability",
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                 yaxis_range=[0,1])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "upgrade_applicability.svg")

### 11. Tenure (have change in second run)

In [70]:
type_list = list(up00['build_existing_model.tenure'].unique())
type_percentage = []
for item in type_list:
    temp = up00.loc[up00['build_existing_model.tenure'] == item]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    type_percentage.append(temp_count)
    
fig = px.bar(x=type_list, y=type_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=240, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title="Tenure",
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                 yaxis_range=[0,1])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / "tenure.svg")